In [1]:
%pip install peft transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 2.7 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM

config = PeftConfig.from_pretrained("giordanorogers/mistral-X-v2")
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1")
model = PeftModel.from_pretrained(model, "giordanorogers/mistral-X-v2")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
config.json: 100%|██████████| 571/571 [00:00<00:00, 2.80MB/s]
model.safetensors.index.json: 100%|██████████| 25.1k/25.1k [00:00<00:00, 21.4MB/s]
model-00001-of-00002.safetensors: 100%|██████████| 9.94G/9.94G [04:31<00:00, 36.6MB/s]
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/huggingface_hub/file_download.py:983: UserWarning: Not enough free disk space to download the file. The expected file size is: 4540.52 MB. The target location /Users/giordanorogers/.cache/huggingface/hub/models--mistralai--Mistral-7B-v0.1/blobs only has 4208.39 MB free disk space.
  warnings.warn(
model-00002-of-00002.safetensors:  89%|████████▉ | 4.06G/4.54G [02:29<00:17, 27.1MB/s]


OSError: [Errno 28] No space left on device

In [ ]:
from transformers import pipeline
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")

generator = pipeline("text-generation", model=model, tokenizer=tokenizer, config=config, )

In [ ]:
# Few-shot examples
few_shot_examples = """
"Food for thought:

The stuff people wrote on the internet from the 90s to the 2020s was the training data for all future AI content.

Most content created after the 2020s will likely just be a repetition of those original 30 years of data.

Crazy."

"New research can now track full body movements of multiple people using only WiFi.

A deep neural network has been developed that maps the phase and amplitude of WiFi signals to specific body regions.

We no longer need cameras to track body movements - just a WiFi signal."

"The mammoth of an AI project that everyone is forgetting about in light of chatGPT:

Google's PaLM (Pathways Language Model) claims to have 560 billion parameters, nearly 3x that of OpenAI's GPT-3 "
"""

# Your actual prompt, appended after the examples
prompt = few_shot_examples + "\nWrite a concise tweet about how much more software developers can accomplish with GitHub Copilot, emphasizing efficiency and AI-powered workflows."

# Generating response with adjusted prompt and specifying max_length
responses = generator(prompt,
                      max_new_tokens=60,
                      num_return_sequences=1,  # Generate a single output
                      do_sample=True,  # Allow for temperature setting
                      temperature=0.8, # Predictable output
                      top_p=0.9,
                      num_beams=5,  # Enable beam search by setting num_beams > 1
                      early_stopping=True,  # Now relevant because we're using beam search
                      length_penalty=0.9) # Encourage concise output

# Print the generated text
for response in responses:
    print(response['generated_text'])
